## Ekstraksi Tapak Bangunan Menggunakan model Modified Vision Transformer (M-VT)

Prosedur ektraksi tapak bangunan menggunakan model M-VT ini disiapkan oleh Tim Peneliti BPN-AI dari Departemen Teknik Geodesi, Universitas Gadjah Mada (UGM).

Terdapat beberapa file yang perlu diunduh terlebih dahulu untuk dapat menjalankan algoritma ekstraksi bangunan ini.

File yang diperlukan dapat diunduh pada: URL

In [1]:
import os
import glob
import matplotlib.pyplot as plt

import torch
import torchvision
from tqdm import tqdm

from imutils import paths

print(torch.__version__)
print(torchvision.__version__)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

# determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

1.12.1
0.13.1
cuda


### Configure Paths and Directories

Pada tahap ini perlu dilakukan konfigurasi direktori untuk mengakses file orthophoto yang akan diekstraksi. Silahkan simpan file othophoto (.tif) di dalam folder: dataset/img_full

In [2]:
GD_PATH = os.getcwd() # get current working directory for the repo
print(GD_PATH)

# Trained model name
model_name = "mvt-bpn-a.pth"
# Trained model path
MODEL_PATH = os.path.join(GD_PATH, "trained_models", model_name)
print(MODEL_PATH)

# Direktori file input foto udara
FULL_IMG_DIR = os.path.join(GD_PATH, "dataset", "img_full")
full_img_dir = os.path.join(FULL_IMG_DIR, "img_sample.tif") #ganti sesuai nama file orthophoto 
print(full_img_dir)

# Direktori file input foto udara setelah dilakukan tiling
FULL_IMG_TILES_DIR = os.path.join(GD_PATH, "dataset", "img_tiles") + "/"

# Direktori file output prediksi segmentasi building footprint
PREDICTIONS_FULL_DIR = os.path.join(GD_PATH, "predictions") + "/" # Lokasi penyimpanan file hasil prediksi model utk segmentasi building footprint

# Direktori file output hasil regularisasi
REGULARIZATION_DIR = os.path.join(GD_PATH, "regularizations") + "/"
print(REGULARIZATION_DIR)

#Tentukan nama file hasil akhir segmentasi building footprint
full_pred_dir = "hasil_tapak_bangunan.tif" 
print(full_pred_dir)

/data/private/BPN_AI/mvt-ekstraksi-bangunan
/data/private/BPN_AI/mvt-ekstraksi-bangunan/trained_models/mvt-bpn-a.pth
/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_full/img_sample.tif
/data/private/BPN_AI/mvt-ekstraksi-bangunan/regularizations/
hasil_tapak_bangunan.tif


### Build Model Architecture

In [3]:
import sys
subfolder = os.path.join(GD_PATH, "models")
sys.path.insert(0, subfolder)

import DCSwin_model

In [4]:
# Load saved model for prediction

print(MODEL_PATH)

model = torch.load(MODEL_PATH) # add MODEL_PATH after training
print("model loaded for prediction")

#model

/data/private/BPN_AI/mvt-ekstraksi-bangunan/trained_models/mvt-bpn-a.pth
model loaded for prediction


In [5]:
torch.cuda.empty_cache()

### Tilling

In [6]:
import cv2
import numpy as np
import patchify as p

def patchData(img, patch_dim:int, step_size:float):
    # pembulatan channel img
    ch1 = np.ceil(img.shape[0]/patch_dim).astype(int)
    ch2 = np.ceil(img.shape[1]/patch_dim).astype(int)

    # menambahkan kolom dan baris kosong pd gambar
    arr0 = np.zeros((ch1*patch_dim, ch2*patch_dim, 3))
    arr0[:img.shape[0], :img.shape[1]] += img
    arr = arr0.astype(np.uint8)

    # patch image
    patch_shape = (patch_dim, patch_dim, 3)
    patches = p.patchify(arr, patch_shape, step=int(patch_dim*step_size))

    img_patches = []
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            img_patches.append(patches[i,j,0,:,:,:])

    return np.array(img_patches)

In [7]:
patch_size = 512 # ukuran patch/tile, dapat diubah/disesuaikan

img = cv2.imread(full_img_dir)

print(f'size ori : {img.shape}')

patch = patchData(img, patch_size, 1)
print(f'patch size : {patch.shape}')
print(patch.shape[0])

for i in range(patch.shape[0]):
    cv2.imwrite(FULL_IMG_TILES_DIR + f'/img_{i+1000}.png', patch[i])

size ori : (3772, 5349, 3)
patch size : (88, 512, 512, 3)
88


### Predicting full images

In [8]:
import random
import gc
from pathlib import Path
import numpy as np
from PIL import Image
import imagesize
import cv2

# PLOTTING PREDICTIONS AS SINGLE IMAGES

# Output folder for the predictions
output_folder_full = PREDICTIONS_FULL_DIR  # check for Windows to save predictions inside the folder

# Short and convert to list
full_images = sorted(list(paths.list_images(FULL_IMG_TILES_DIR)))

# Prediction Threshold
THRESHOLD = 0.5

# PLOT TEST IMAGES as RGB
for n in range(len(full_images)):
  gc.collect()
  # Test image number
  testImgName = str(Path(full_images[n]).stem) + '.tif'
  #IMG_WIDTH, IMG_HEIGHT = imagesize.get(testImgName)
    #print('#', testImgName)

   # Make predicton on a test image specified with counter n
  test_img = full_images[n]
  test_img_input = np.expand_dims(test_img, 0)
  #print('#', test_img_input[0])

  # PyTorch --> works
  model.eval()
  with torch.no_grad():
    image = cv2.imread(test_img_input[0])
    image = cv2.resize(image, dsize = (patch_size, patch_size), interpolation=cv2.INTER_CUBIC)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float32") / 255
    
    # print('SIZE: ', image.shape)

    # make the channel axis to be the leading one, add batch dimension
    image = np.transpose(image, (2, 0, 1))
    # create a PyTorch tensor
    image = np.expand_dims(image, 0)
    # flash the tensor to the device
    image  = torch.from_numpy(image).to(DEVICE)

    # make the prediction
    predMask = model(image).squeeze()
    # pass result through sigmoid
    predMask = torch.sigmoid(predMask)

    # convert result to numpy array
    predMask = predMask.cpu().numpy()

    # filter out the weak predictions and convert them to integers
    predMask = (predMask > THRESHOLD) * 255
    predMask = predMask.astype(np.uint8)

    # generate image from array
    pIMG = Image.fromarray(predMask)
    pIMG.save(str(output_folder_full + testImgName))

    #print('Prediction:', testImgName, 'saved to:', output_folder_full)

print("Prediction saved to:", output_folder_full)

Prediction saved to: /data/private/BPN_AI/mvt-ekstraksi-bangunan/predictions/


In [9]:
output_folder = PREDICTIONS_FULL_DIR + "/" + "*.tif"

predictions = glob.glob(output_folder)
predictions.sort()

### BUILDING FOOTPRINT REGULARIZATION

In [10]:
# DEFINE NECESSARY PATHS FOR REGULARIZATION PART

projectRegDir = os.path.join(GD_PATH, "projectRegularization")
print(projectRegDir)

ptw = os.path.join(projectRegDir, "pretrained_weights") 
print(ptw)

# GET THE PATHS FOR TRAINED GAN MODELS
ENCODER = os.path.join(ptw, "E140000_e1")
GENERATOR = os.path.join(ptw, "E140000_net")

print(ENCODER)
print(GENERATOR)

/data/private/BPN_AI/mvt-ekstraksi-bangunan/projectRegularization
/data/private/BPN_AI/mvt-ekstraksi-bangunan/projectRegularization/pretrained_weights
/data/private/BPN_AI/mvt-ekstraksi-bangunan/projectRegularization/pretrained_weights/E140000_e1
/data/private/BPN_AI/mvt-ekstraksi-bangunan/projectRegularization/pretrained_weights/E140000_net


In [11]:
# CREATE A NEW variables.py WITH USERS PATHS

folder_path = projectRegDir
file_name = 'variables.py'
file_path = os.path.join(folder_path, file_name)

with open(file_path, 'w') as f:
    f.write('# CONFIGURE THE PATHS HERE: \n\n')
    # f.write('# TRAINING \n')
    # f.write('DATASET_RGB = ' + '"' + str(TRAIN_IMG_DIR + '*.tif' + '"') + '\n')
    # f.write('DATASET_GTI = ' + '"' + str(TRAIN_MASK_DIR + '*.tif' + '"') + '\n')
    # f.write('DATASET_SEG = ' + '"' + str(PREDICTIONS_DIR + '*.tif' + '"') + '\n')
    f.write('\n')
    f.write('DEBUG_DIR = ' + '"' + str('./debug/') + '"' + '\n')
    f.write('\n')
    f.write('# INFERENCE \n')
    f.write('INF_RGB = ' + '"' + str(FULL_IMG_TILES_DIR + '*.png' + '"') + '\n')
    f.write('INF_SEG = ' + '"' + str(PREDICTIONS_FULL_DIR + '*.tif' + '"') + '\n')
    f.write('INF_OUT = ' + '"' + str(REGULARIZATION_DIR + '"') + '\n')
    f.write('\n')
    f.write('MODEL_ENCODER = ' + '"' + str(ENCODER) + '"' + '\n')
    f.write('MODEL_GENERATOR = ' + '"' + str(GENERATOR) + '"' + '\n')
    f.close()
 
print("variables.py created with users paths...")


variables.py created with users paths...


#### Run projectRegularization

Perlu penyesuaian pada command di bawah dengan mengubah absolute path dari file regularize.py

In [12]:
!python /data/private/BPN_AI/mvt-ekstraksi-bangunan/projectRegularization/regularize.py

/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_tiles/img_1000.png /data/private/BPN_AI/mvt-ekstraksi-bangunan/predictions/img_1000.tif
Regularization: 100%|█████████████████████████████| 2/2 [00:05<00:00,  2.59s/it]
/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_tiles/img_1001.png /data/private/BPN_AI/mvt-ekstraksi-bangunan/predictions/img_1001.tif
Regularization: 100%|█████████████████████████████| 8/8 [00:01<00:00,  7.61it/s]
/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_tiles/img_1002.png /data/private/BPN_AI/mvt-ekstraksi-bangunan/predictions/img_1002.tif
Regularization: 100%|███████████████████████████| 11/11 [00:01<00:00,  7.83it/s]
/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_tiles/img_1003.png /data/private/BPN_AI/mvt-ekstraksi-bangunan/predictions/img_1003.tif
Regularization: 100%|█████████████████████████████| 7/7 [00:01<00:00,  3.61it/s]
/data/private/BPN_AI/mvt-ekstraksi-bangunan/dataset/img_tiles/img_1004.png /data/private/BPN_AI/mvt-

In [13]:
# Read Regularizations to plot and compare results

regularizations = glob.glob(REGULARIZATION_DIR + "*.tif")
regularizations.sort()

print("Jumlah predicted images: ", len(predictions))
print("Jumlah regularized images: ", len(regularizations))

Jumlah predicted images:  88
Jumlah regularized images:  88


Code to plot RGB, GT, PREDICTION and REGULARIZATION in a single plot for comparison.

Change parameter n accordingly.

In [14]:
# n = 10

# fig = plt.figure(figsize=(18,12))
# ax1 = fig.add_subplot(141)

# ax1.set_title('RGB: ')
# image = cv2.imread(test_images[n])[:,:,::-1]
# ax1.imshow(image)
# ax1.set_axis_off()

# ax2 = fig.add_subplot(142)
# ax2.set_title('Ground truth: ')
# image = cv2.imread(test_masks[n])[:,:,::-1]
# #image *= 255
# ax2.imshow(image)
# ax2.set_axis_off()

# ax3 = fig.add_subplot(143)
# ax3.set_title('Prediction: ')
# image = cv2.imread(predictions[n])[:,:,::-1]
# ax3.imshow(image)
# ax3.set_axis_off()

# ax4 = fig.add_subplot(144)
# ax4.set_title('Regularization: ')
# image = cv2.imread(regularizations[n])[:,:,::-1]
# ax4.imshow(image)
# ax4.set_axis_off()

# # DEFINE PATH FOR PLOTS TO BE SAVED
# figPath = GD_PATH + "/" + "plots" + "/" "compare-" + str(n) + ".png"
# print(figPath)

# # # Save plot
# # fig.savefig(figPath)

### Combine patches into one full prediction image

In [15]:
import cv2

original_image = cv2.imread(full_img_dir)

height_ori, width_ori, channels = original_image.shape

result_height = height_ori + (patch_size - (height_ori-(patch_size*int(height_ori/patch_size))))
result_width = width_ori + (patch_size - (width_ori-(patch_size*int(width_ori/patch_size))))

# print(width_ori)
# print(height_ori)
# print(result_width)
# print(result_height)

In [16]:
from PIL import Image

# Create a blank result image with the specified dimensions
result_image = Image.new("I", (result_width, result_height))

# List the image files in the folder and sort them numerically
image_files = sorted(list(paths.list_images(REGULARIZATION_DIR)))

# Initialize coordinates for pasting images in the result image
x, y = 0, 0

# Iterate through the image files and paste them into the result image
for image_file in image_files:
    image = Image.open(image_file)
    
    # Resize the image to fit within the specified dimensions
    image.thumbnail((result_width, result_height))
    
    # Paste the resized image into the result image at the current position
    result_image.paste(image, (x, y))
    
    # Update the x-coordinate for the next image
    x += image.width
    
    # If the x-coordinate exceeds the result_width, reset it and move to the next row
    if x >= result_width:
        x = 0
        y += image.height

# Save the resulting single image
result_image.save(full_pred_dir)

#### Set Geotransform

In [17]:
from osgeo import gdal

# Open the source raster dataset
src_dataset = gdal.Open(full_img_dir, gdal.GA_ReadOnly)

# Get the geotransform from the source dataset
geotransform = src_dataset.GetGeoTransform()

# Close the source dataset
src_dataset = None

# Open a new or existing target raster dataset
# Replace 'target.tif' with the path to your target dataset
dst_dataset = gdal.Open(full_pred_dir, gdal.GA_Update)

# Set the geotransform for the target dataset
dst_dataset.SetGeoTransform(geotransform)

# Close the target dataset to save changes
dst_dataset = None